In [ ]:
!pip uninstall spark-stratifier

In [ ]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=bd73d989fe7076bcc42ec657b99ca78d5003a8e8cc5e9fb9369f8e27420a8fda
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
!pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable


# News categorization model development

## Logistic regression

In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [2]:
"""conf=SparkConf().setAppName('NewsCatagorization')\
.set("spark.driver.memory", '30g')\
.set("spark.executor.memory", "30g")\
.set("spark.executor.memoryOverhead", "50g")\
.set("spark.task.cpus","8")\
.set("spark.executor.cores", "8")\
.set("spark.executor.instances","16")

sc=SparkContext(conf=conf)
sc
"""

'conf=SparkConf().setAppName(\'NewsCatagorization\').set("spark.driver.memory", \'30g\').set("spark.executor.memory", "30g").set("spark.executor.memoryOverhead", "50g").set("spark.task.cpus","8").set("spark.executor.cores", "8").set("spark.executor.instances","16")\n\nsc=SparkContext(conf=conf)\nsc\n'

In [2]:
conf = SparkConf().setAppName('NewsCategorization') \
    .set("spark.driver.memory", '8g') \
    .set("spark.executor.memory", "8g") \
    .set("spark.executor.memoryOverhead", "2g") \
    .set("spark.task.cpus", "2") \
    .set("spark.executor.cores", "2") \
    .set("spark.executor.instances", "3")

sc = SparkContext(conf=conf)
print(sc)

24/05/31 12:26:29 WARN Utils: Your hostname, starias-T14s resolves to a loopback address: 127.0.1.1; using 10.152.86.208 instead (on interface wlp0s20f3)
24/05/31 12:26:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/31 12:26:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/05/31 12:26:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


<SparkContext master=local[*] appName=NewsCategorization>


In [3]:
spark=SparkSession(sc)
spark

In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer

from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import col, trim, regexp_replace,split,expr,lit

# Load data
df = spark.read.csv("input/news.csv", header=True, inferSchema=True)

In [5]:
df.rdd.getNumPartitions()

8

In [6]:
num_partitions=4*8
df= df.repartition(num_partitions)

In [7]:
#!pip install scikit-learn

In [8]:
#from xgboost.spark import SparkXGBClassifier

In [9]:
df.count()

1716608

In [10]:
from pyspark.sql.functions import col, desc,when,split,format_number

In [11]:
from pyspark.sql.functions import split

# Create a new DataFrame with description_filtered as arrays
df= df.withColumn('description_filtered', split(col('description_filtered'), ' '))
# Show the new DataFrame
df.show(truncate=False)


+--------------+-----------------------------------------------------------------------------------------------------------------------------+
|category_label|description_filtered                                                                                                         |
+--------------+-----------------------------------------------------------------------------------------------------------------------------+
|1.0           |[james, augustus, henry, murray, take, office, nugget, yesteryear, wind, 134132, 2, ots]                                     |
|2.0           |[peter, kalmus, climate, scientist, author, change, live, well, spark, climate, revolution]                                  |
|2.0           |[respond, environmental, crisis, good, planning, jeff, sparrow, writes, extract, book]                                       |
|2.0           |[remarkable, turnaround, sea, cow, favorite, food, mosquito, lagoon, mean, emergency, feeding, program, end]                 |

In [12]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier,MultilayerPerceptronClassifier,NaiveBayes,LinearSVC,DecisionTreeClassifier,OneVsRest

In [ ]:
# Calculate the fraction for each class
fractions = df.groupBy("category_label").count().withColumnRenamed("count", "total").collect()
total_count = df.count()
fractions_dict = {row['category_label']: row['total'] / total_count for row in fractions}
fractions_dict

{0.0: 0.03125,
 1.0: 0.03125,
 3.0: 0.03125,
 2.0: 0.03125,
 7.0: 0.03125,
 4.0: 0.03125,
 6.0: 0.03125,
 5.0: 0.03125,
 8.0: 0.03125,
 11.0: 0.03125,
 10.0: 0.03125,
 9.0: 0.03125,
 12.0: 0.03125,
 14.0: 0.03125,
 13.0: 0.03125,
 15.0: 0.03125,
 16.0: 0.03125,
 18.0: 0.03125,
 19.0: 0.03125,
 17.0: 0.03125,
 20.0: 0.03125,
 23.0: 0.03125,
 21.0: 0.03125,
 22.0: 0.03125,
 24.0: 0.03125,
 25.0: 0.03125,
 28.0: 0.03125,
 27.0: 0.03125,
 26.0: 0.03125,
 29.0: 0.03125,
 31.0: 0.03125,
 30.0: 0.03125}

In [14]:
# Function to evaluate model and get best parameters
def evaluate_model(model, model_name):
    print('Training the model')

    # Train the model using cross-validation
    fitted_model = model.fit(train_set)

    # Get the best model from cross-validation
    best_model = fitted_model.stages[-1].bestModel

    print('Making predictions on the training set')
    # Make predictions on the training set
    train_predictions = fitted_model.transform(train_set)

    print('Making predictions on the test set')
    # Make predictions on the test set
    test_predictions = fitted_model.transform(test_set)

    # Initialize the evaluator
    evaluator = MulticlassClassificationEvaluator(labelCol="category_label", predictionCol="prediction", metricName="accuracy")

    print('Evaluating the model on training set')
    # Evaluate the model on the training set
    train_accuracy = evaluator.evaluate(train_predictions)

    print('Evaluating the model on test set')
    # Evaluate the model on the test set
    test_accuracy = evaluator.evaluate(test_predictions)

    print(f"{model_name} Train Accuracy: {train_accuracy}")
    print(f"{model_name} Test Accuracy: {test_accuracy}")

    # Print the best parameters
    print(f"Best parameters for {model_name}:")
    
    for param, value in best_model.extractParamMap().items():
        print(f"  {param.name}: {value}")

    return train_accuracy, test_accuracy

In [ ]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import numpy as np



# Define the HashingTF and IDF stages
hashingTF = HashingTF(inputCol="description_filtered", outputCol="rawFeatures", numFeatures=10000)
idf = IDF(inputCol="rawFeatures", outputCol="features")

# Define the classifiers
lr = LogisticRegression(labelCol="category_label", featuresCol="features")
rf = RandomForestClassifier(labelCol="category_label", featuresCol="features")
nb = NaiveBayes(labelCol="category_label", featuresCol="features")

# Define parameter grids
paramGrid_nb = ParamGridBuilder() \
    .addGrid(nb.smoothing, [0.5, 1.0, 2.0]) \
    .build()



reg_values = np.logspace(-4, 4, num=100)

l1_ratios = np.linspace(0, 1, num=10) 

paramGrid_lr = ParamGridBuilder() \
    .addGrid(lr.regParam, reg_values) \
    .addGrid(lr.elasticNetParam, l1_ratios) \
    .build()



paramGrid_rf = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 50, 100]) \
    .addGrid(rf.maxDepth, [5, 10, 20]) \
    .build()

# Create cross validators
cv_nb = CrossValidator(estimator=nb, estimatorParamMaps=paramGrid_nb,
                       evaluator=MulticlassClassificationEvaluator(labelCol="category_label", predictionCol="prediction", metricName="accuracy"),
                       numFolds=3, parallelism=3)

cv_lr = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid_lr,
                       evaluator=MulticlassClassificationEvaluator(labelCol="category_label", predictionCol="prediction", metricName="accuracy"),
                       numFolds=3, parallelism=3)

cv_rf = CrossValidator(estimator=rf, estimatorParamMaps=paramGrid_rf,
                       evaluator=MulticlassClassificationEvaluator(labelCol="category_label", predictionCol="prediction", metricName="accuracy"),
                       numFolds=3)

# Create pipelines
pipeline_nb = Pipeline(stages=[hashingTF, idf, cv_nb])
pipeline_lr = Pipeline(stages=[hashingTF, idf, cv_lr])
pipeline_rf = Pipeline(stages=[hashingTF, idf, cv_rf])

# Split data
(train_set, test_set) = df.randomSplit([0.8, 0.2], seed=0)



# Evaluate models using the evaluate_model function
results = {}
for model_pipeline, model_name in [
    #(pipeline_nb, "Naive Bayes"),
    (pipeline_lr, "Logistic Regression"),
    #(pipeline_rf, "Random Forest"),
]:
    print(f'Evaluating {model_name}')
    train_accuracy, test_accuracy = evaluate_model(model_pipeline, model_name)
    results[model_name] = {"train_accuracy": train_accuracy, "test_accuracy": test_accuracy}
    
    

Evaluating Logistic Regression
Training the model


24/05/31 12:48:04 ERROR RetryingBlockTransferor: Exception while beginning fetch of 1 outstanding blocks (after 2 retries)
java.io.IOException: Connecting to /10.152.86.208:35853 failed in the last 4750 ms, fail this connection directly
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:210)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:131)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.transferAllOutstanding(RetryingBlockTransferor.java:173)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.lambda$initiateRetry$0(RetryingBlockTransferor.java:206)
	at java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:511)
	at java.util.concurrent.FutureTask.run(FutureTask.java:266)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:6

24/05/31 12:48:53 ERROR RetryingBlockTransferor: Exception while beginning fetch of 1 outstanding blocks (after 1 retries)
java.io.IOException: Connecting to /10.152.86.208:35853 failed in the last 4750 ms, fail this connection directly
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:210)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:131)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.transferAllOutstanding(RetryingBlockTransferor.java:173)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.lambda$initiateRetry$0(RetryingBlockTransferor.java:206)
	at java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:511)
	at java.util.concurrent.FutureTask.run(FutureTask.java:266)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:6

24/05/31 12:53:03 ERROR RetryingBlockTransferor: Exception while beginning fetch of 1 outstanding blocks (after 3 retries)
java.io.IOException: Connecting to /10.152.86.208:35853 timed out (120000 ms)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:291)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:214)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:131)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.transferAllOutstanding(RetryingBlockTransferor.java:173)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.lambda$initiateRetry$0(RetryingBlockTransferor.java:206)
	at java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:511)
	at java.util.concurrent.FutureTask.run(FutureTask.java:266)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.ut

24/05/31 12:55:08 ERROR RetryingBlockTransferor: Exception while beginning fetch of 1 outstanding blocks (after 1 retries)
java.io.IOException: Connecting to /10.152.86.208:35853 timed out (120000 ms)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:291)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:214)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:131)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.transferAllOutstanding(RetryingBlockTransferor.java:173)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.lambda$initiateRetry$0(RetryingBlockTransferor.java:206)
	at java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:511)
	at java.util.concurrent.FutureTask.run(FutureTask.java:266)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.ut

24/05/31 12:59:18 ERROR RetryingBlockTransferor: Exception while beginning fetch of 1 outstanding blocks (after 3 retries)
java.io.IOException: Connecting to /10.152.86.208:35853 timed out (120000 ms)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:291)
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:214)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:131)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.transferAllOutstanding(RetryingBlockTransferor.java:173)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.lambda$initiateRetry$0(RetryingBlockTransferor.java:206)
	at java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:511)
	at java.util.concurrent.FutureTask.run(FutureTask.java:266)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.ut

24/05/31 13:01:23 ERROR RetryingBlockTransferor: Exception while beginning fetch of 1 outstanding blocks (after 1 retries)
java.io.IOException: Connecting to /10.152.86.208:35853 failed in the last 4750 ms, fail this connection directly
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:210)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:131)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.transferAllOutstanding(RetryingBlockTransferor.java:173)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.lambda$initiateRetry$0(RetryingBlockTransferor.java:206)
	at java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:511)
	at java.util.concurrent.FutureTask.run(FutureTask.java:266)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:6

24/05/31 13:03:28 ERROR RetryingBlockTransferor: Exception while beginning fetch of 1 outstanding blocks (after 1 retries)
java.io.IOException: Connecting to /10.152.86.208:35853 failed in the last 4750 ms, fail this connection directly
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:210)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:131)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.transferAllOutstanding(RetryingBlockTransferor.java:173)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.lambda$initiateRetry$0(RetryingBlockTransferor.java:206)
	at java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:511)
	at java.util.concurrent.FutureTask.run(FutureTask.java:266)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:6

24/05/31 13:05:33 ERROR RetryingBlockTransferor: Exception while beginning fetch of 1 outstanding blocks 
java.io.IOException: Connecting to /10.152.86.208:35853 failed in the last 4750 ms, fail this connection directly
	at org.apache.spark.network.client.TransportClientFactory.createClient(TransportClientFactory.java:210)
	at org.apache.spark.network.netty.NettyBlockTransferService$$anon$2.createAndStart(NettyBlockTransferService.scala:131)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.transferAllOutstanding(RetryingBlockTransferor.java:173)
	at org.apache.spark.network.shuffle.RetryingBlockTransferor.start(RetryingBlockTransferor.java:152)
	at org.apache.spark.network.netty.NettyBlockTransferService.fetchBlocks(NettyBlockTransferService.scala:151)
	at org.apache.spark.network.BlockTransferService.fetchBlockSync(BlockTransferService.scala:102)
	at org.apache.spark.storage.BlockManager.fetchRemoteManagedBuffer(BlockManager.scala:1172)
	at org.apache.spark.storage.BlockMa

In [ ]:
#sc.stop()

In [ ]:
!pip install scikit-learn

In [10]:
import numpy as np
# Select the required column
df_features_labels = df.select("description_filtered", "category_label")

# Convert to Pandas DataFrame for compatibility with scikit-learn
pandas_df = df_features_labels.toPandas()

In [13]:
pandas_df

,description_filtered,category_label
0,grocer recall prepared food onion tied salmone...,0.0
1,fitch affirms sabic outlook stable,0.0
2,global pert market growth prospect insight ana...,0.0
3,global tung oil market strategic assessment re...,0.0
4,fxcm pro partner devexperts provide liquidity ...,0.0
...,...,...
1716603,selena gomez development new tv show described...,31.0
1716604,selena gomez developing new television set dis...,31.0
1716605,hurricane irma hour angle already hit million ...,31.0
1716606,googleorg invests 1 million serve create latin...,31.0


In [17]:
u=pandas_df['description_filtered'].tolist()
u=np.array(u)

MemoryError: Unable to allocate 40.0 GiB for an array with shape (1716608,) and data type <U6248

In [12]:
# Extract features (X) and labels (y)
X = np.array(pandas_df['description_filtered'].tolist())
y = pandas_df['category_label'].values

# Collect features and labels
#features_labels = df_transformed.select("features", "category_label").collect()

MemoryError: Unable to allocate 40.0 GiB for an array with shape (1716608,) and data type <U6248

In [5]:
from pyspark.ml.feature import CountVectorizer, IDF, Tokenizer,HashingTF
from pyspark.ml import Pipeline

# Tokenizer to split the text into words
tokenizer = Tokenizer(inputCol="description_filtered", outputCol="words")

# CountVectorizer to convert text into a matrix of token counts
#vectorizer = CountVectorizer(inputCol="words", outputCol="rawFeatures")

num_features = 2**18

# HashingTF to convert text into a matrix of token counts
hashing_tf = HashingTF(inputCol="words", outputCol="rawFeatures", numFeatures=num_features)

# IDF to transform the count matrix to a normalized tf-idf representation
idf = IDF(inputCol="rawFeatures", outputCol="features")

# Pipeline to execute the steps
pipeline = Pipeline(stages=[tokenizer, hashing_tf, idf])

# Fit the pipeline to the DataFrame
model = pipeline.fit(df)

# Transform the DataFrame to get the features
df_transformed = model.transform(df)


In [6]:
df_transformed.show()

24/05/27 22:22:35 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


+--------------+--------------------+--------------------+--------------------+--------------------+
|category_label|description_filtered|               words|         rawFeatures|            features|
+--------------+--------------------+--------------------+--------------------+--------------------+
|           0.0|grocer recall pre...|[grocer, recall, ...|(262144,[7194,435...|(262144,[7194,435...|
|           0.0|fitch affirms sab...|[fitch, affirms, ...|(262144,[8578,250...|(262144,[8578,250...|
|           0.0|global pert marke...|[global, pert, ma...|(262144,[3968,814...|(262144,[3968,814...|
|           0.0|global tung oil m...|[global, tung, oi...|(262144,[8145,166...|(262144,[8145,166...|
|           0.0|fxcm pro partner ...|[fxcm, pro, partn...|(262144,[43204,63...|(262144,[43204,63...|
|           0.0|water bottle mark...|[water, bottle, m...|(262144,[8145,181...|(262144,[8145,181...|
|           0.0|solarwinds ceo ke...|[solarwinds, ceo,...|(262144,[69703,82...|(262144,[697

In [18]:
import numpy as np
# Select the required columns
df_features_labels = df_transformed.select("features", "category_label")

# Convert to Pandas DataFrame for compatibility with scikit-learn
pandas_df = df_features_labels.toPandas()



# Collect features and labels
#features_labels = df_transformed.select("features", "category_label").collect()

24/05/27 22:40:20 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7001c5322c00>>
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 

KeyboardInterrupt



In [20]:
# Extract features (X) and labels (y)
X = pandas_df['description_filtered'].tolist()
y = pandas_df['category_label'].values

In [23]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [27]:
# Initialize HashingVectorizer
num_features = 1000  # Number of features (adjust as needed)
hashing_vectorizer = HashingVectorizer(n_features=num_features)

# Transform the text data to a matrix of token counts
X_hashed = hashing_vectorizer.transform(X)

# Initialize TfidfTransformer
tfidf_transformer = TfidfTransformer()

# Transform the token counts to TF-IDF representation
X_tfidf = tfidf_transformer.fit_transform(X_hashed)

# Convert the sparse matrix to a dense matrix if needed
#X_tfidf_dense = X_tfidf.toarray()

# Print the transformed features and labels
print("Features (X):", X_tfidf)
print("Labels (y):", y)

Features (X):   (0, 119)	0.37782184895348164
  (0, 223)	0.3391235897269139
  (0, 414)	0.289078028690946
  (0, 586)	-0.3586289023237469
  (0, 746)	0.3455405618475319
  (0, 821)	0.3645115364307206
  (0, 850)	-0.36019888904328323
  (0, 952)	0.3847789824682703
  (1, 22)	-0.5120927452407951
  (1, 497)	-0.43333623286050665
  (1, 526)	-0.4385721263195585
  (1, 609)	0.4190474719352858
  (1, 840)	0.42665493767469526
  (2, 65)	-0.26713891555915836
  (2, 96)	0.2918124569517634
  (2, 157)	0.26948567442972904
  (2, 158)	0.2105510043769622
  (2, 327)	0.22911812683825664
  (2, 365)	-0.24891786971452565
  (2, 414)	0.21919526806619602
  (2, 492)	0.24675371427892087
  (2, 502)	0.248481858012751
  (2, 522)	0.24726822194757517
  (2, 552)	0.20407683997015152
  (2, 679)	0.25954132470356356
  :	:
  (1716604, 903)	0.18057904782117315
  (1716604, 930)	-0.2502975234964976
  (1716605, 468)	0.4250842709017103
  (1716605, 493)	0.38489466592947336
  (1716605, 508)	-0.37879752260362043
  (1716605, 660)	0.37814468093

In [36]:
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.svm import SVC

# Define the evaluation function
def evaluate_learners(models, X, y):
    train_scores = []
    test_scores = []
    for model in models:
        cv_results = cross_validate(model, X, y, cv=5, return_train_score=True)
        train_scores.append(cv_results['train_score'])
        test_scores.append(cv_results['test_score'])
    return train_scores, test_scores

# Load a sample dataset, e.g., Iris dataset
#iris = datasets.load_iris()
#X, y = iris.data, iris.target

# Define the parameter grid for SVC
param_grid = {
    'C': [0.1, 1, 10, 100]
}

# Create a base SVC model
svc = SVC(kernel='linear')
svc

SVC(kernel='linear')

In [37]:
# Instantiate the grid search model
grid_search = GridSearchCV(estimator=svc, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search

GridSearchCV(cv=5, estimator=SVC(kernel='linear'), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100]}, scoring='accuracy')

In [ ]:
# Fit the grid search to the data
grid_search.fit(X_tfidf, y)

24/05/28 07:06:34 WARN TransportChannelHandler: Exception in connection from /10.152.184.160:46169
java.io.IOException: Connection timed out
	at sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:223)
	at sun.nio.ch.IOUtil.read(IOUtil.java:192)
	at sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:379)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:254)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1132)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:357)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:151)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:788)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeysOptimized(NioEventLoop.java:724)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeys(NioEventLoop.j

In [ ]:
# Get the best model
best_svc_model = grid_search.best_estimator_
models = [best_svc_model]

In [ ]:
# Evaluate the model
train_scores, test_scores = evaluate_learners(models, X, y)

# Print the results
print("Best model parameters:", grid_search.best_params_)
print("Training scores:", train_scores)
print("Test scores:", test_scores)